In [2]:
import os
import pygpc
import numpy as np
import h5py
from collections import OrderedDict
from matplotlib import pyplot as plt

In [13]:
fn_results = os.path.join(os.getcwd(), "data")

## Define model
model = pygpc.testfunctions.Ishigami()

## Define parameters
parameters = OrderedDict()
parameters["x1"] = pygpc.Beta(pdf_shape=[1, 1], pdf_limits=[-np.pi, np.pi])
parameters["x2"] = pygpc.Beta(pdf_shape=[1, 1], pdf_limits=[-np.pi, np.pi])
parameters["x3"] = pygpc.Beta(pdf_shape=[1, 1], pdf_limits=[-np.pi, np.pi])
parameters["a"] = 7.0
parameters["b"] = 0.1

# Define problem
problem = pygpc.Problem(model, parameters)

# gPC options
options = dict()
options["solver"] = "LarsLasso"
options["settings"] = None
options["order_start"] = 3
options["order_end"] = 15
options["seed"] = 1
options["projection"] = False
options["order_max_norm"] = 1.
options["interaction_order"] = 2
options["matrix_ratio"] = 2
options["n_cpu"] = 0
options["gradient_enhanced"] = False
options["gradient_calculation"] = "standard_forward"
options["error_type"] = "loocv"
options["error_norm"] = "absolute"
options["n_samples_validation"] = 1e2
options["qoi"] = 0
options["classifier"] = "learning"
options["n_samples_discontinuity"] = 10
options["adaptive_sampling"] = True
options["eps"] = 0.03
options["n_grid_init"] = 100
options["GPU"] = False
options["fn_results"] = fn_results
options["classifier_options"] = {"clusterer": "KMeans",
                                 "n_clusters": 2,
                                 "classifier": "MLPClassifier",
                                 "classifier_solver": "lbfgs"}

# define algorithm
algorithm = pygpc.MERegAdaptiveProjection(problem=problem, options=options)

# Initialize gPC Session
session = pygpc.Session(algorithm=algorithm)

# Run gPC session
session, coeffs, results = session.run()

# # Post-process gPC
# pygpc.get_sensitivities_hdf5(fn_gpc=options["fn_results"],
#                              output_idx=None,
#                              calc_sobol=True,
#                              calc_global_sens=True,
#                              calc_pdf=True,
#                              algorithm="sampling",
#                              n_samples=1e4)

# with h5py.File(fn_results + "_validation.hdf5", "r") as f:
#     val_coords_norm = f["grid/coords_norm"][:]
#     val_coords = f["grid/coords"][:]
#     val_results = f["model_evaluations/original_all_qoi"][:]

# # Validate gPC vs original model function (2D-slice)
# pygpc.validate_gpc_plot(session=session,
#                         coeffs=coeffs,
#                         random_vars=["w_ein_pc", "w_iin_pc"],
#                         coords=val_coords,
#                         data_original=val_results,
#                         output_idx=0,
#                         fn_out=session.fn_results + '_val',
#                         n_cpu=session.n_cpu)

Creating initial grid (Random) with n_grid=100
Performing 100 initial simulations!
It/Sub-it: 3/2 Performing simulation 001 from 100 [                                        ] 1.0%


Total function evaluation: 0.0025069713592529297 sec
Determining gPC approximation for QOI #0:
Determining gPC domains ...
Refining domain boundary ...
Performing 10 simulations to refine discontinuity location!
It/Sub-it: Domain boundary/N/A Performing simulation 01 from 10 [====                                    ] 10.0%
Total function evaluation: 0.00257110595703125 sec
Updating classifier ...
Determine gPC coefficients using 'LarsLasso' solver ...
Determine gPC coefficients using 'LarsLasso' solver ...
LOOCV 01 from 25 [=                                       ] 4.0%
LOOCV 02 from 25 [===                                     ] 8.0%
LOOCV 03 from 25 [====                                    ] 12.0%
LOOCV 04 from 25 [======                                  ] 16.0%
LOOCV 05 from 25 [========                                ] 20.0%
LOOCV 06 from 25 [=========                               ] 24.0%
LOOCV 07 from 25 [===========                             ] 28.0%
LOOCV 08 from 25 [=========